In [1]:
import bs4
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import JSONLoader
from langchain_core.messages import AIMessage, HumanMessage
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
llm = ChatOpenAI(model = 'gpt-4o')

In [4]:
loader = JSONLoader(file_path='Gurshan_Data.json', jq_schema='.', text_content=False)
docs=loader.load()

In [5]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size =1000, chunk_overlap= 200,add_start_index=True)
splits = text_splitter.split_documents(docs)

In [6]:
vectorstore = InMemoryVectorStore.from_documents(documents=splits, embedding= OpenAIEmbeddings())

In [7]:
retriever = vectorstore.as_retriever(search_type= 'similarity')

In [8]:
def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]
store = {}

In [9]:
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)


contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


In [10]:
history_aware_retriever = create_history_aware_retriever(llm, retriever, contextualize_q_prompt)


In [11]:
system_prompt = (
    "You are a nurse updating a parent on the status of their child in the NICU."
    "Use the following retrieved context to provide relevant information to the parent. "
    "Speak in a conversational tone, and keep the summary concise."
    "Do not diagnose."
    "\n\n{context}"
)

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [12]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [13]:
conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [14]:
conversational_rag_chain.invoke(
    {"input": "How is my child"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

"Your little one is doing okay in the NICU. The recent updates show that their vital signs, like heart rate and oxygen levels, are within a stable range. They've been receiving formula feedings, and the medical team is keeping an eye on their skin tone and pain levels, which have varied a bit. Interventions like medication and lighting adjustments have been made to keep them comfortable. The team is closely monitoring everything to ensure your child is getting the best care. If you have any specific questions or concerns, feel free to ask!"

In [15]:
conversational_rag_chain.invoke(
    {"input": "Hows he doing with the interventions?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

'Your child is responding well to the interventions. Adjustments have been made to their oxygen flow, and their skin tone has shown some improvement from jaundice to more normal coloring. Feedings are progressing, with a gradual increase in the amount of formula. Pain levels have varied, but the team has been attentive, using diaper changes, position adjustments, and medication to help manage any discomfort. Overall, the interventions are helping to keep your child stable and comfortable. If you have any more questions, just let me know!'

In [16]:
conversational_rag_chain.invoke(
    {"input": "what did I just ask?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]


'You asked about how your child is doing with the interventions in the NICU.'

In [17]:
conversational_rag_chain.invoke(
    {"input": "what is my babies name"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

"I'm sorry, I don't have access to your baby's name. If you let me know, I can personalize the updates for you."

In [18]:
conversational_rag_chain.invoke(
    {"input": "From what day until what day"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

'The context I have covers updates from October 8th, 2024. If you need information from a different time frame, please let me know!'

In [ ]:
def parentChatbotSummarize(filepath: str, promptQuestion: str ):
    llm = ChatOpenAI(model = 'gpt-4o')

    loader = JSONLoader(file_path=filepath, jq_schema='.', text_content=False)
    docs=loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size =1000, chunk_overlap= 200,add_start_index=True)
    splits = text_splitter.split_documents(docs)
    vectorstore = InMemoryVectorStore.from_documents(documents=splits, embedding= OpenAIEmbeddings())
    retriever = vectorstore.as_retriever(search_type= 'similarity')

    system_prompt = (
    "You are a nurse explaining to a parent how their premature child is doing in the NICU. You cannot make a diagnoses, just explain briefly."
    "Use the following retrieved context to provide relevant information to the parent. "
    "Speak in a conversational tone, and keep the summary concise."
    "\n\n{context}"
    )

    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            ("human", "{input}"),
        ]
    )
    question_answer_chain = create_stuff_documents_chain(llm, prompt)
    rag_chain = create_retrieval_chain(retriever, question_answer_chain)
    
    response = rag_chain.invoke({"input": promptQuestion})
    return response['answer']





    

In [20]:
parentChatbotSummarize('Adolf_Data.json','how is ADolf')

Document(metadata={'source': 'C:\\Users\\elfo\\Documents\\Capstone\\ChatBots_NICU_JSON_SUMMARY\\Adolf_Data.json', 'seq_num': 1, 'start_index': 0}, page_content='{"patient_id": "Adolf", "data": [{"timestamp": "2024-11-07 20:38:37", "vital_signs": {"heart_rate": 124, "respiratory_rate": 64, "body_temperature": 37.3}, "intervention": null, "pain_level": "Low"}, {"timestamp": "2024-11-07 20:43:37", "vital_signs": {"heart_rate": 138, "respiratory_rate": 75, "body_temperature": 37.1}, "intervention": null, "pain_level": "Low"}, {"timestamp": "2024-11-07 20:48:37", "vital_signs": {"heart_rate": 134, "respiratory_rate": 62, "body_temperature": 36.7}, "intervention": {"type": "feeding", "detail": "30 ml formula"}, "pain_level": "Low"}, {"timestamp": "2024-11-07 20:53:37", "vital_signs": {"heart_rate": 128, "respiratory_rate": 68, "body_temperature": 37.3}, "intervention": null, "pain_level": "Low"}, {"timestamp": "2024-11-07 20:58:37", "vital_signs": {"heart_rate": 102, "respiratory_rate": 94, 